In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [20]:
base_cols = None
box_score = box_scores[0]

# for box_score in box_scores
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")
    
    # total of the stats for basic and advanced (last row)
    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals.index = totals.index.str.lower()
    # highest mark of each of the columns for basic and advanced (all rows except last row)
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"
    
    summary = pd.concat([totals, maxes])
    
    if base_cols is None:
        base_cols = list(summary.index.drop_duplicates(keep="first"))
        base_cols = [b for b in base_cols if "bpm" not in b]
        
    summary = summary[base_cols]
    
    summaries.append(summary)
# summary = pd.concat(summaries, axis=1)

C:\Users\admin\AppData\Local\Temp\ipykernel_22276\670034871.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
C:\Users\admin\AppData\Local\Temp\ipykernel_22276\385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
C:\Users\admin\AppData\Local\Temp\ipykernel_22276\385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
C:\Users\admin\AppData\Local\Temp\ipykernel_22276

In [21]:
summary

mp          240.000
mp          240.000
fg           37.000
fga          82.000
fg%           0.451
             ...   
blk%_max      7.100
tov%_max     57.100
usg%_max     33.800
ortg_max    258.000
drtg_max    121.000
Length: 70, dtype: float64